# __PostgreSQL NOTES__

This is a python notebook but I will try to mirror SQL syntax to the best of my ability

## LOADING DATA INTO PostgreSQL

In [ ]:
# A. NORMAL LOADING FROM .sql FILE
#How to do:

(Right click on) Database - Create -> Database - Load Database file
#database file = folder containing all relevant ******.sql******** files

(Right click on) (Newly Created Database) - Query Tool...

#Query Tool used to 'query' information from files - the main thing of SQL (below)

------------------------------------

# B. TABLE CREATION - THE MAIN WAY TO DO BUSINESS!

#alternatively, can create own table:

CREATE TABLE table_name (
    some_column_name TYPE column_constraint,
    column_2_name TYPE, 
    column_3_name TYPE column constraint
    );

# 1. TYPE = TEXT or INTEGER or NUMERIC 
# or VARCHAR(100) (number of max characters, regardless of type) or DATE - check internet for more!

# ***********TIP!!!!!!!!!******** unless you want a specific format that SQL will conform to (like DATE),
# it is ALWAYS a good idea to identify the TYPE by varchar(50) - it avoids type errors for importing,
# which will LIKELY happen for data that is not 100% clean (which is always the case)

# if choosing varchar from a dropdown list (and using "length" as the number for max characters),
# it will be "character varying"
***
# do ___NOT___ choose character varying[] - that creates an array for SQL, and because csv data is not in arrays,
# it will trigger an error!

--------------------------------------

# 2. column_constraint can be:
#  a. boolean (e.g.):
    ... price numeric CHECK (price > 0) # yields true and null values
    # i.e., yields positive prices
    
    price numeric CHECK (price > 0),
    discounted_price numeric CHECK (discounted_price > 0),
    CHECK (price > discounted_price)
    ); # multiple booleans across columns, with third CHECK not corresponding
    # to any one particular column
    
#  b.  column constraint NOT NULL 
name text NOT NULL,
price numeric NOT NULL CHECK (price > 0) # combines both

# in a query, you use this syntax:
select * from calendar
where price is not null
order by 4 DESC;

#  c.  UNIQUE
product_no integer UNIQUE,
[...]
# UNIQUE shows only unique items within that column (i.e., takes out duplicates
# within that column)

# Can also combine...
CREATE TABLE example (
    a integer, 
    b integer, 
    c integer, 
    UNIQUE (a, c)
);
# This says: "There must be no duplicates of **combinations** of specific a and c values"
# Can have duplicates of a and duplicates of c, but not particular duplicates of 
# a and c:
e.g.:
    12 24
    12 12
    12 24
    22 24
# row 3 would be taken out with UNIQUE (a, c) but not the others

# ******NOTE!: Once a table is created, the order of the columns CANNOT be modified.
# What you can do is rename all of the columns before populating with data 
# by right clicking on each column and modifying until you are satisfied with the order

#  d.  PRIMARY KEY = same as UNIQUE NOT NULL on a column
# can also do PRIMARY KEY (a, c) too

# a few other constraints that can be done!

# C. IMPORT DATA FROM .csv FILE - HOW-TO!

1. Create Table manually under Query Tool:
    CREATE TABLE (follow Create Table syntax above, i.e., listing column name and type)
    - Populating the newly created table via the query tool requires:
        COPY table_name(column_name1, column_name2, column_name3...#write out ****ALL columns you want to populate***)
        FROM 'C:\Users\jafon\Documents\PythonMaterials\Data\AirBnB_Listings\listings.csv' with DELIMITER ',' csv HEADER; 
        
    
2. Import Data into newly created table
(Under Browser): Database - Schemas - public - Tables (right click) -> Create -> Table
 - Name the Table (something different than the create Table under the query)
 - Under Columns - populate the column names by clicking on the dropdown, selecting the table you just
created via the query (or I guess you could manually write out the columns here - You can skip step 1 if you do this)
 - Click save and create the table 
 - Access this newly table under the browser -> (right click) -> Import/Export -> Choose Impport
 - Select location of dataset file
 - Header YES or ON, DELIMITER ','
 - Save and Import


POTENTIAL ERRORS TO LOADING DATA (and their solutions):
-- "There is no such file or directory" or "permission denied":
     --- There is a permission error from Windows blocking pgAdmin from accessing the file folder
     --- right-click on the folder that contains the desired file -> Properties -> Change user permissions
            
-- "extra data after last expected column":
     --- The number of columns being imported does not match up with the number of columns
     --- listed in the COPY table_name(...) query
            
---**********"IMPORTANT"****: You MUST import ***ALL*** of the columns from the csv - you cannot pick and choose!
      --- There are two ways to fix this:
      1. Deleting unnecessary columns in the original csv dataset - saving a new copy and loading from there
      ---- This is called "preprocessing" the data - and it is the easiest way
                
      2. Import ALL columns FIRST from csv, THEN modify the table
      a. ALTER TABLE table_Name DROP COLUMN column5
      ALTER TABLE table_Name DROP COLUMN column2 (etc.)
                    
      b. This may be too process-heavy/cumbersome for super large datasets
      You can skip this by creating and intermediary table AND THEN 
      picking and choosing from there:
                    
CREATE temporary table t (x1 integer, ... , x10 text)
Copy from the file into it:

COPY t (x1, ... , x10)
FROM '/path/to/my_file'
WITH  (format csv)

INSERT INTO my_table (x2, x5, x7, x10)
SELECT x2, x5, x7, x10
FROM t

DROP TABLE t;

                        
# ^^^^^NOTE!!!: This is a good method for re-ordering columns too!
*****
The failure to easily choose column order and modify tables 
is a severe limitation of postgreSQL/pgAdmin. Other sql''s (e.g., MySQL) allow you to 
pick and choose columns normally.
                        
**********************How to alter columns
# altering a table is simple - just do an ALTER TABLE call
ALTER TABLE table_name
ALTER COLUMN column_name TYPE new_desiredcolumn type

# if that doesn't work, do this
ALTER TABLE table_name
ALTER COLUMN column_name TYPE new_desiredcolumn type
USING column_name::new_desired column type

# example
ALTER TABLE calendar
ALTER COLUMN price TYPE MONEY NOT NULL
USING price::MONEY


# FINAL NOTE***************
# double dash (--) is used for comments in PostgreSQL

## MAIN QUERY TOOLS

In [ ]:

1. SELECT _____ FROM _____;

# Select often the first thing to do, unless doing FROM (thing), SELECT...
# first space is what you want to look at, i.e., Column names
# Use SELECT * if you want to look at whole table
# second space is name of SQL file you want to look at
#semicolon at end doesn't do anything - just conventional
# example:
     SELECT temperature, clouds_and_sun FROM weather;
    
# while both can be syntactically accepted, 
# it is convention to do multiple rows

SELECT
    *
FROM
    stations;
    
# Can also change column names

SELECT
    temperature temp
FROM
    weather;
# can also do temperature AS temp (but typing the word AS is unnecessary)

2. WHERE
# purpose = can filter upon boolean conditions

# a.  use BETWEEN for ranges:
a BETWEEN x AND y # equivalent to a >= x AND a <= y
a NOT BETWEEN x AND y # equivalent to a < x OR a > y

# b.  use LIKE for matching strings:
"abc" LIKE "abc"    true
'abc' LIKE 'a%'     true # % = matches any preceding or following sequence of
                         # 0 or more characters
'abc' LIKE '_b_'    true # _ = matches any single character
'abc' LIKE 'c'      false
# b. use SIMILAR TO for string matching via regular expressions
# regex in a different noteset...

3. ORDER BY
# purpose = applies order of values

4. LIMIT
# purpose = returns set maximum number of values, e.g., "three longest trips"
# good to do generally if your computer is having a difficult time loading a 
# dataset, e.g., use LIMIT 10000;
SELECT
    trip_id,
    start_date,
    duration
FROM
    trips
WHERE
    bike_id = 27,
    zip_code = 94107 AND
    subscriber_type LIKE "Customer" #always double quotes in SQL
ORDER BY duration DESC,
start_date ASC # can order columns sequentially, i.e., if the duration column has duplicates, go to start_date to sort
LIMIT 3;


In [ ]:
# for reference - not a note!
COPY listings(name, host_name, neighbourhood, latitude, longitude, room_type, price, 
reviews_per_month, availability_365, lastreview, host_count, id, hostid, 
numberofreviews, minimumnights)
from 'C:\Users\jafon\Documents\PythonMaterials\Data\AirBnB_Listings\listings.csv' with delimiter ',' csv header; 

## AGGREGATING AND GROUPING

In [ ]:
# AGGREGATING means can be done via similar NumPy methods
SELECT
    city, 
    AVG(lat) AS latitude,
    AVG(long) AS longitude,
    MAX(time),
    COUNT(*) AS station_count
FROM
    stations;
    
#GROUPING gets rids of duplicates, similar to UNIQUE in creating a table
SELECT
    city, 
    lat AS latitude,
    long AS longitude,
    MAX(time),
    COUNT(*) AS station_count
FROM
    stations
GROUP BY 1, 2, 3; # grouping by city, latitude, and longitude. 
# Can write out names of grouping columns too, but numbers are easier
# NOTE***: Cannot group aggregated columns (as there is now only a 
# single value for that column)
# NOTE***: THEREFORE, must group ALL OTHER columns!!!

## JOINS

In [ ]:
# JOINS join MULTIPLE TABLES together
# In this instance, MUST identify the table from which a specific column comes from
SELECT
    trips.trip_id,
    trips.start_station,
    stations.lat,
    stations.long
FROM
    trips 
JOIN
    stations
ON
    trips.start_station = stations.name; # where to do the join
# note that the values of the two ONs ***MUST BE THE SAME TO DO THE JOIN***
# (the column names need not be the same - just the values of those columns)
# See example in CTEs below on how to get around this (do multiple joins across tables)
    
# Can also use "aliases" to shorten table names. The above is the same as:
SELECT
    t.trip_id,
    t.start_station,
    s.lat,
    s.long
FROM
    trips t
JOIN
    stations s
ON
    t.start_station = s.name;
    
# TYPES OF JOINS
1. INNER JOIN  - returns only matching columns, dropping everything else 
# SQL default^^^, i.e., same as JOIN
2. OUTER JOIN - returns everything, leaving non-matches as null values
# not recommended, as may choke up computer when viewing large databases
3. LEFT JOIN - returns everything on left table, leaves non-matches on 
right table as null values
4. RIGHT JOIN - returns everything on right table, leaves non-matches on 
left table as null values
# more difficult to read since we read left to right - 
# better to flip around the tables in the query and do a left join instead


## COMMON TABLE EXPRESSIONS (CTEs)

In [ ]:
# CTE is joining a table on a previously processed query,
# (as opposed to joining two tables)

# Important to do because aggregation functions happen AFTER joins occur

CTE = WITH 
    Name of CTE to refer to later
AS (
    first query
)
SELECT
    second query
);

# EXAMPLE to break down:
WITH
    locations
AS (
    -- A simple query to get the averages of lat and long on a city level.
    SELECT
        city,
        AVG(lat) lat,
        AVG(long) long
    FROM
        stations
    GROUP BY 1
)

-- Joining the locations table we created with the trips table to count trips.
SELECT
    l.city,
    l.lat,
    l.long,
    COUNT(*)
FROM
    locations l

-- We need an intermediate join to go from locations to stations 
-- because the trips table does not have a "city" column.
JOIN
    stations s
ON
    l.city = s.city
JOIN
    trips t
ON
    t.start_station = s.name
GROUP BY 1,2,3;

# BREAKDOWN OF EXAMPLE:
'''
Under *locations*, after finding the averages of lat and long coordinates
for the stations of every city,
I want to join those values onto trips table.
HOWEVER, because trips table does not have a city name column,
I must join locations.city onto stations.city,
THEN DO A SECOND JOIN of trips.start_station onto stations.name.

Then, due to an inner join, the cities will be grouped with the averages of 
the coordinates, i.e., averages of lat, long, and count of number of trips
per city will be outputted.

'''

## CASE WHEN

In [ ]:
# Similar to WHERE - Allows for manipulation of values within a NEW column
CASE WHEN _condition_ THEN _value1_ ELSE _value2_ END AS _new_column_name_

# example:
SELECT
    (CASE WHEN dockcount > 20 THEN 'large' ELSE 'small' END) station_size,
    COUNT(*) as station_count
FROM 
    stations
GROUP BY 1;
'''
station_size is a new column, yielding *large* or *small* values based on
the dockcount, and the query is grouped by large and small.
'''
# Can do multiple conditions too:
SELECT player_name,
       weight,
       CASE WHEN weight > 250 THEN 'over 250'
            WHEN weight > 200 THEN '201-250'
            WHEN weight > 175 THEN '176-200'
            ELSE '175 or under' END AS weight_group
FROM benn.college_football_players
# don't need to do BETWEENs since the query is analyzed top-down!
# May still want to do that for clarity purposes...

In [ ]:
# for project

CREATE TABLE listings (
    id BIGINT,
    name TEXT,
    host_id BIGINT,
    host_name TEXT,
    neighbourhood_group TEXT,
    neighbourhood TEXT,
    latitude TEXT,
    longitude TEXT,
    room_type TEXT,
    price REAL,
    minimum_nights INTEGER,
    number_of_reviews INTEGER,
    last_review DATE,
    reviews_per_month REAl,
    calculated_host_listings_count INTEGER,
    availability_365 TEXT
);

CREATE TABLE calendar (
	listing_id BIGINT,
	calender_date DATE,
	available CHAR,
	price TEXT
);

CREATE TABLE reviews (
	listing_id BIGINT,
	id BIGINT,
	review_date DATE,
	reviewer_id BIGINT,
	reviewer_name TEXT,
	comments TEXT
);

CREATE TABLE listingstest (
    id VARCHAR(100),
    name VARCHAR(100),
    host_id VARCHAR(100),
    host_name VARCHAR(100),
    neighbourhood VARCHAR(100),
    latitude VARCHAR(100),
    longitude VARCHAR(100),
    room_type VARCHAR(100),
    price VARCHAR(100),
    minimum_nights VARCHAR(100),
	availability_365 VARCHAR(100),
    number_of_reviews VARCHAR(100),
    last_review VARCHAR(100),
    calculated_host_listings_count VARCHAR(100),
	reviews_per_month VARCHAR(100)
	);


code for thinkful project:

1. What is the most expensive listing? What else can you tell me about the listing?

select * from listingstest
order by price DESC
limit 5

ANSWER: The most expensive listing in San Francisco is priced at $9,999 per night. Called a "Test Apartment", it is an entire
apartment  in the Russian Hill neighborhood. It requires a 30-day minimum stay and is available about half the year. According to the number of reviews (zero), no one has taken advantage of this listing.

2. What neighborhoods seem to be the most popular?

select
neighbourhood,
avg(number_of_reviews) as numrev,
avg(reviews_per_month) as revrate 
from listingstest
where reviews_per_month is not null
group by neighbourhood
order by 3 desc;

ANSWER: The 10 most popular neighborhoods, in order from most popular, are: Presidio, Visitacion Valley, Outer Sunset, Parkside, Outer Richmond, Diamond Heights, Bayview, Ocean View, Excelsior, and Crocker Amazon. Because there was no transactional metric for usage and therefore popularity, popularity was instead determined by the highest average of reviews submitted per month.

 3. What time of year is the cheapest time to go to your city? What about the busiest?

ANSWER 3a:

select calender_date, avg(price) as avgprice 
from calendar
group by 1
order by 2 asc
limit 50;

ANSWER 3a: Out of the top 50 hits for the dataset time period of Dec. 2018 to Dec. 2019, 
all but 6 hits were in March and April. 5 of the remaining six were in december and january (the last one being in May)

ANSWER 3b: 

SELECT 
DATE_TRUNC('month', calender_date) m,
--DATE_TRUNC is a great method for compiling timestamp type columns by desired field
--(in this case, by month)
COUNT(listing_id) AS count,
-- ^ representing the total count for that particular month
SUM(CASE WHEN available = 't' THEN 1 ELSE 0 END) AS tsum,
-- ^ representing the total occupied, or "taken", count for that particular month
SUM(CASE WHEN available = 'f' THEN 1 ELSE 0 END) AS fsum,
-- ^ representing the total available, or "free", count for that particular month
(SUM(CASE WHEN available = 't' THEN 1 ELSE 0 END) * 100.0 / count(listing_id)) AS tratio
-- Yes, you can perform arithmetic operations in PostgreSQL
-- Make sure, however, that you write out the ENTIRE operation;
-- You ***CANNOT*** use newly created columns as part of the operation
-- Also, for percentages, use "100.0" instead of "100" - 100 rounds to an integer while 
-- float gives you the decimals
from calendar
GROUP BY m
-- As a reminder, when grouping, ALWAYS make sure you are grouping by ALL
-- non-aggregated columns that are called.
ORDER BY tratio desc;

ANSWER 3b: I use a column "tratio" to help me determine on a ratio basis what the busiest time of the year is. According to that column, the busiest times of the year is the month of February. The next most busiest months are March, April, May, and January, in that order.


In [ ]:
CREATE TABLE calendar (
	listing_id BIGINT,
	calender_date DATE,
	available CHAR,
	price REAL
);

CREATE TABLE reviews (
	listing_id BIGINT,
	id BIGINT,
	review_date DATE,
	reviewer_id BIGINT,
	reviewer_name TEXT,
	comments TEXT
);

-- Note: the type values for listings that were recommended by Thinkful caused errors for me
-- while varying character is not what I was looking for, 
-- it was the only thing that helped me move forward with this assignment

CREATE TABLE listingstest (
    id VARCHAR(100),
    name VARCHAR(100),
    host_id VARCHAR(100),
    host_name VARCHAR(100),
    neighbourhood VARCHAR(100),
    latitude VARCHAR(100),
    longitude VARCHAR(100),
    room_type VARCHAR(100),
    price MONEY,
    minimum_nights VARCHAR(100),
	availability_365 VARCHAR(100),
    number_of_reviews VARCHAR(100),
    last_review VARCHAR(100),
    calculated_host_listings_count VARCHAR(100),
	reviews_per_month VARCHAR(100)
	);
